# Search Engine

This lab is about starting from scratch, and hence having to structure the code yourself. A specification for the program you are to implement is given below - do pay attention as misunderstandings may cost you marks! Being precise is a necessary skill for a programmer. To give an executive summary, you are to code a search engine for recipes. A data set has been provided. The search engine is to be pretty basic, returning all recipes that contain all of the provided keywords. However, the user can choose from a number of orderings depending on their food preferences, which you need to support.

## Marking and submission

These lab exercises are marked, and contribute to your final grade. This lab exercise has 20 marks to earn, equivalent to 12% of your final grade.

Please submit your completed workbook to the auto marker before 2021-11-14 20:00 GMT. The workbook you submit must be an .ipynb file, which is saved into the directory you're running Jupyter; alternatively you can download it from the menu above using `File -> Download As -> Notebook (.ipynb)`. Remember to save your work regularly (`Save and checkpoint` in the `File` menu, the icon of a floppy disk, or `Ctrl-S`). It is wise to verify it runs to completion with _Restart & Run All_ before submission.

You must comply with the universities plagiarism guidelines: http://www.bath.ac.uk/library/help/infoguides/plagiarism.html

## Specification

The system must provide a function ``search``, with the following specification:
```
def search(query, ordering = 'normal', count = 10):
  ...
```

It `print`s out the results of the search, subject to the following rules:
1. It selects from the set of all recipes that contain __all__ of the words in the query (the positions/order of the words in the recipe are to be ignored).
2. It orders them based on the provided ordering (a string, meaning defined below).
3. It `print`s the top `count` matches only, preserving the order from best to worst. Must `print` just their title, one per line. Must be less than `count` if the search is specific enough that less than `count` recipes match.

As an aside, do not worry about memory usage. If duplicating some data can make your code faster/neater then feel free.



### Data set

A file, `recipes.json` is provided, containing 17K recipes. It can be parsed into a Python data structure using the [`json`](https://docs.python.org/3/library/json.html) module. It is a list, where each recipe is a dictionary containing various keys:
* `title` : Name of recipe; you can assume these are unique
* `categories` : A list of tags assigned to the recipe
* `ingredients` : What is in it, as a list
* `directions` : List of steps to make the recipe
* `rating` : A rating, out of 5, of how good it is
* `calories` : How many calories it has
* `protein` : How much protein is in it
* `fat` : How much fat is in it

Note that the data set was obtained via web scrapping and hence is noisy - every key except for `title` is missing from at least one recipe. Your code will need to cope with this.

You will probably want to explore the data before starting, so you have an idea of what your code has to deal with.

Data set came from https://www.kaggle.com/hugodarwood/epirecipes/version/2 though note it has been cleaned it up, by deleting duplicates and removing the really dodgy entries.



### Search

The search should check the following parts of the recipe (see data set description below):
* `title`
* `categories`
* `ingredients`
* `directions`

For instance, given the query "banana cheese" you would expect "Banana Layer Cake with Cream Cheese Frosting" in the results. Note that case is to be ignored ("banana" matches "Banana") and the words __do not__ have to be next to one another, in the same order as the search query or even in the same part of the recipe ("cheese" could appear in the title and "banana" in the ingredients). However, all words in the search query __must__ appear somewhere.



### Tokenisation

This is the term for breaking a sentence into each individual word (token). Traditionally done using regular expressions, and Python does have the `re` module, but there is no need to do that here (`re` can be quite fiddly). For matching words your tokenisation must follow the following steps:
1. Convert all punctuation and digits into spaces. For punctuation use the set in [`string.punctuation`](https://docs.python.org/3/library/string.html#string.punctuation), for digits [`string.digits`](https://docs.python.org/3/library/string.html#string.digits). You may find [`translate()`](https://docs.python.org/3/library/stdtypes.html#str.translate) interesting!
2. [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) to extract individual tokens.
3. Ignore any token that is less than $3$ characters long.
4. Make tokens lowercase.

When matching words for search (above) or ordering (below) it's only a match if you match an entire token. There are many scenarios where this simple approach will fail, but it's good enough for this exercise. The auto marker will be checking the above is followed! When doing a search the code should ignore terms in the search string that fail the above requirements.



### Ordering

There are three ordering modes to select from, each indicated by passing a string to the `search` function:
* `normal` - Based simply on the number of times the search terms appear in the recipe. A score is calculated and the order is highest to lowest. The score sums the following terms (repeated words are counted multiple times, i.e. "cheese cheese cheese" is $3$ matches to "cheese"):
    * $8 \times$ Number of times a query word appears in the title
    * $4 \times$ Number of times a query word appears in the categories
    * $2 \times$ Number of times a query word appears in the ingredients
    * $1 \times$ Number of times a query word appears in the directions
    * The `rating` of the recipe (if not available assume $0$)

* `simple` - Tries to minimise the complexity of the recipe, for someone who is in a rush. Orders to minimise the number of ingredients multiplied by the numbers of steps in the directions.

* `healthy` - Order from lowest to highest by this cost function:
$$\frac{|\texttt{calories} - 510n|}{510} + 2\frac{|\texttt{protein} - 18n|}{18} + 4\frac{|\texttt{fat} - 150n|}{150}$$
Where $n \in \mathbb{N}^+$ is selected to minimise the cost ($n$ is a positive integer and $n=0$ is not allowed). This can be understood in terms of the numbers $510$, $18$ and $150$ being a third of the recommended daily intake (three meals per day) for an average person, and $n$ being the number of whole meals the person gets out of cooking/making the recipe. So this tries to select recipes that neatly divide into a set of meals that are the right amount to consume for a healthy, balanced diet. Try not to overthink the optimisation of $n$, as it's really quite simple to do!

To clarify the use of the ordering string, to get something healthy that contains cheese you might call `search('cheese', 'healthy')`. In the case of a recipe that is missing a key in its dictionary the rules are different for each search mode:
* `normal` - Consider a missing entry in the recipe (e.g. no `ingredients` are provided) to simply mean that entry can't match any search words (because it has none!), but the item is still eligible for inclusion in the results, assuming it can match the search with a different entry.
* `simple` - If a recipe is missing either `ingredients` or `directions` it is dropped from such a search result. Because the data is messy if either of these lists is of length $1$ it should be assumed that the list extraction has failed and the recipe is to also be dropped from the search results.
* `healthy` - If any of `calories`, `protein` or `fat` is missing the recipe should be dropped from the result.



### Extra

You may find the [_inverted index_](https://en.wikipedia.org/wiki/Inverted_index) interesting. It's a data structure used by search engines. For each word a user may search for this contains a list of all documents (recipes) that contain the word. This may take a little effort to understand, but the resulting code will be both faster and neater.

## Advice

* Don't just start coding: Make a plan and work out what you intend to do.
* Think about structure, as messy code leads to mistakes.
* Plan your data structures. Don't be afraid to use sheets of paper if that works for you!
* Don't duplicate code, put it in a function/method instead.

* Divide and conquer. Break the system into parts that can implemented with minimal dependency on the rest. Functions or OOP are both suitable for doing this.
* Test early. Verify the individual parts work before trying to combine them. Factor this into the order you implement the parts of the system - don't implement something you are going to struggle to test before implementing, and verifying, other parts.
* Do not try and do a 'big bang', where you get it all working at once. Instead, get it working with features missing, then add those features in, one at a time.

* Keep things as simple as possible. Avoid long functions/methods.
* Include comments, as a form of planning and for your own sanity!
* Regularly reset the kernel and rerun the entire workbook. It is very easy to break something but not notice, because the correct version remains in memory.

## Marks
* __20 marks__: For many different _unit tests_ that will be run on `search`. They will cover all of the details in the above specification!
    * One test checks that it's faster than $0.1$ seconds on average (on the computer which runs the auto marker, which is quite fast) to do a search, so try not to be too inefficient (it ignores any time your notebook spends building data structures to be used by `search`). Note that the validation implementation comes in at $0.001$ seconds per search (after $5.5$ seconds of preparation), so this is very achievable!
    * You may want to look into Python's `set()` object, as it is useful for _one_ of the possible ways to implement this.
    * There will be sorting. The [Sorting how to](https://docs.python.org/3/howto/sorting.html) may help.
    * The auto marker does give some feedback, and you can run it as many times as you want. Don't be afraid to test an incomplete or semi-broken version of your code if stuck or unsure. It may help!
    * The validation implementation is 104 lines of code split over 5 cells (including white space for clarity and comments). Coded by someone who probably has much more experience than you, so you shouldn't aim to match this, but it's a good clue: If you find yourself at 500 lines you may want to stop and think some more! (line count does not include testing code, which is about the same amount again)

In [294]:
import string
import pandas as pd
import numpy as np
from collections import Counter

def clean_list(l):
    try:
        return ",".join(map(str, l)).replace(",", " ")
    except TypeError:
        return np.nan

def solution(emblem, user_entered):
    return not (Counter(emblem) - Counter(user_entered))


def read():

    df = pd.read_json("recipes.json")
    
    return df

def tokenise():
    
    df = read()

    df.loc[:, 'title_t_lst'] = df['title'].astype(str).str.replace(f"[{string.punctuation + string.digits}]", " ", regex=True).apply(str.lower).str.findall("\w{3,}").str.join(" ").str.split(' ')
    df.loc[:, 'ingredients_t_lst'] = df['ingredients'].apply(clean_list).astype(str).str.replace(f"[{string.punctuation + string.digits}]", " ", regex=True).apply(str.lower).str.findall("\w{3,}").str.join(" ").str.split(' ')
    df.loc[:, 'categories_t_lst'] = df['categories'].apply(clean_list).astype(str).str.replace(f"[{string.punctuation + string.digits}]", " ", regex=True).apply(str.lower).str.findall("\w{3,}").str.join(" ").str.split(' ')
    df.loc[:, 'directions_t_lst'] = df['directions'].apply(clean_list).astype(str).str.replace(f"[{string.punctuation + string.digits}]", " ", regex=True).apply(str.lower).str.findall("\w{3,}").str.join(" ").str.split(' ')

    df.loc[:, "title_t_c"] = [Counter(i) for i in df[f'title_t_lst']]
    df.loc[:, "ingredients_t_c"] = [Counter(i) for i in df[f'ingredients_t_lst']]
    df.loc[:, "categories_t_c"] = [Counter(i) for i in df[f'categories_t_lst']]
    df.loc[:, "directions_t_c"] = [Counter(i) for i in df[f'directions_t_lst']]

    df['tokens_set'] = (df['ingredients_t_lst'] + df['title_t_lst'] + df['directions_t_lst'] + df['categories_t_lst'])
    df['tokens_set'] = df['tokens_set'].apply(set).apply(list).apply(clean_list)

    return df


def count_occur(df, query):
    
    df['m'] = True
    for w in query.split(' '):
        df.loc[:,'m'] &= df["tokens_set"].str.contains(f' {w} ')

    df = df[df['m']].copy().reset_index(drop=True)

    df.loc[:, 'title_count'] = [sum([df['title_t_c'][r].get(k, 0) for k in query.split(' ')]) for r in range(len(df))]
    df.loc[:, 'ingredients_count'] = [sum([df['ingredients_t_c'][r].get(k, 0) for k in query.split(' ')]) for r in range(len(df))]
    df.loc[:, 'categories_count'] = [sum([df['categories_t_c'][r].get(k, 0) for k in query.split(' ')]) for r in range(len(df))]
    df.loc[:, 'directions_count'] = [sum([df['directions_t_c'][r].get(k, 0) for k in query.split(' ')]) for r in range(len(df))]

    return df


def score(df, count, ordering):

    if ordering == 'normal':
        df.loc[:, "score"] = 8 * df["title_count"] + 4 * df["categories_count"] + 2 * df["ingredients_count"] + df["directions_count"] + df["rating"]
        df = df.sort_values(by=["score"], ascending=False)
        df = df.reset_index(drop=True)
        df = df.iloc[0:count]


    if ordering == 'simple':
        df = df.dropna(subset=["ingredients", "directions"]).copy()
        df = df[df["directions"].str.len() > 1]

        df.loc[:, "score"] = df["ingredients"].str.len() * (df["directions"].str.len())
        df = df.sort_values(by=["score"], ascending=True)
        df = df.reset_index(drop=True)
        df = df.iloc[0:count]

    if ordering == 'healthy':
        df = df.copy()
        df = df.dropna(subset=["calories", "protein", "fat"])

        df.loc[:, "score"] = (abs(df["calories"] - 510) / 510) +  (abs(df["protein"] - 18) / 9) + (2 * abs(df["fat"] - 150) / 75)

        df = df.sort_values(by=["score"], ascending=True)
        df = df.reset_index(drop=True)
        df = df.iloc[0:count]
    
    return df

In [295]:
read_in = tokenise()

def search(query, ordering="normal", count=10):
    """Assumes an input of keywords to search for seperated by a space."""

    query = pd.Series(query)
    query = query.str.replace(f"[{string.punctuation + string.digits}]", " ", regex=True)
    query = query.apply(str.lower)
    query = query.str.findall("\w{3,}").str.join(" ")[0]
    
    if len(query) == 0:
        return

    df = count_occur(read_in, query)
    df = score(df, count, ordering)
    
    recipe = list(df["title"])
    print("\n".join(recipe))
        
    return df


In [296]:
search('peanut butter', 'normal', 7)

Chocolate Cake with Milk Chocolate-Peanut Butter Frosting and Peanut Butter Brittle 
Peanut Butter Tart with Caramel-Peanut Glaze 
Peanut Butter Cake with Chocolate-Peanut Butter Icing 
Peanut Butter Banana Cream Pie 
Banana Cupcakes with Peanut Butter Frosting 
Chocolate-Peanut Butter Cake with Cream Cheese and Butterfinger Frosting 
Peanut Butter Cheesecake with Peanut Brittle 


,title,categories,ingredients,directions,rating,calories,protein,fat,title_t_lst,ingredients_t_lst,...,ingredients_t_c,categories_t_c,directions_t_c,tokens_set,m,title_count,ingredients_count,categories_count,directions_count,score
0,Chocolate Cake with Milk Chocolate-Peanut Butt...,"[Cake, Mixer, Chocolate, Dessert, Bake, Valent...","[Nonstick vegetable oil spray, 3 cups all purp...",[Line large rimmed baking sheet with foil; bru...,3.750,1380.0,20.0,69.0,"[chocolate, cake, with, milk, chocolate, peanu...","[nonstick, vegetable, oil, spray, cups, all, p...",...,"{'nonstick': 1, 'vegetable': 3, 'oil': 3, 'spr...","{'cake': 1, 'mixer': 1, 'chocolate': 1, 'desse...","{'line': 2, 'large': 3, 'rimmed': 1, 'baking':...",bring thick each water pan airtight separate h...,True,4,4,1,3,50.750
1,Peanut Butter Tart with Caramel-Peanut Glaze,"[Milk/Cream, Food Processor, Mixer, Nut, Desse...","[1 1/4 cups all purpose flour, 3 tablespoons s...",[Blend first 3 ingredients in processor to com...,3.750,446.0,6.0,33.0,"[peanut, butter, tart, with, caramel, peanut, ...","[cups, all, purpose, flour, tablespoons, sugar...",...,"{'cups': 1, 'all': 1, 'purpose': 1, 'flour': 1...","{'milk': 1, 'cream': 2, 'food': 1, 'processor'...","{'blend': 1, 'first': 1, 'ingredients': 1, 'pr...",thick freeze water pan airtight hours dessert ...,True,3,5,1,7,48.750
2,Peanut Butter Cake with Chocolate-Peanut Butte...,"[Cake, Chocolate, Nut, Dessert, Bake, Quick & ...","[1 18.25-ounce package yellow cake mix, 2 1/2 ...",[Preheat oven to 350°F. Grease and flour 2 9-i...,4.375,668.0,15.0,35.0,"[peanut, butter, cake, with, chocolate, peanut...","[ounce, package, yellow, cake, mix, cups, crea...",...,"{'ounce': 1, 'package': 1, 'yellow': 1, 'cake'...","{'cake': 1, 'chocolate': 1, 'nut': 1, 'dessert...","{'preheat': 1, 'oven': 1, 'grease': 1, 'and': ...",diameter icing dessert cool centers oven and s...,True,4,2,1,4,48.375
3,Peanut Butter Banana Cream Pie,"[Milk/Cream, Food Processor, Mixer, Dessert, B...","[6 ounces vanilla wafer cookies, 6 tablespoons...",[Preheat oven to 350°F. Combine all ingredient...,5.000,537.0,6.0,34.0,"[peanut, butter, banana, cream, pie]","[ounces, vanilla, wafer, cookies, tablespoons,...",...,"{'ounces': 2, 'vanilla': 3, 'wafer': 1, 'cooki...","{'milk': 1, 'cream': 2, 'food': 1, 'processor'...","{'preheat': 1, 'oven': 2, 'combine': 2, 'all':...",spoonful arrange hours diameter warm back than...,True,2,5,2,7,46.000
4,Banana Cupcakes with Peanut Butter Frosting,"[Cake, Mixer, Egg, Breakfast, Dessert, Bake, K...","[1 1/4 cups all purpose flour, 1 1/2 teaspoons...",[Position rack in center of oven and preheat t...,4.375,NaN,NaN,NaN,"[banana, cupcakes, with, peanut, butter, frost...","[cups, all, purpose, flour, teaspoons, baking,...",...,"{'cups': 2, 'all': 1, 'purpose': 1, 'flour': 1...","{'cake': 1, 'mixer': 1, 'egg': 1, 'breakfast':...","{'position': 1, 'rack': 2, 'center': 2, 'oven'...",sour each airtight among dessert cool pescatar...,True,2,4,3,4,44.375
5,Chocolate-Peanut Butter Cake with Cream Cheese...,"[Cake, Mixer, Chocolate, Nut, Dessert, Bake, K...","[2 1/4 cups heavy whipping cream, 1/2 cup (pac...","[Bring cream and sugar to simmer in saucepan, ...",4.375,973.0,14.0,58.0,"[chocolate, peanut, butter, cake, with, cream,...","[cups, heavy, whipping, cream, cup, packed, go...",...,"{'cups': 3, 'heavy': 2, 'whipping': 2, 'cream'...","{'cake': 1, 'mixer': 1, 'chocolate': 1, 'nut':...","{'bring': 1, 'cream': 4, 'and': 7, 'sugar': 5,...",bring pan among hours diameter parchment desse...,True,2,6,1,7,43.375
6,Peanut Butter Cheesecake with Peanut Brittle,"[Cake, Nut, Dessert, Bake, Kid-Friendly, Cream...","[Nonstick vegetable oil spray, 2 8-ounce packa...",[Preheat oven to 325°F. Spray 9-inch-diameter ...,3.750,320.0,9.0,25.0,"[peanut, butter, cheesecake, with, peanut, bri...","[nonstick, vegetable, oil, spray, ounce, packa...",...,"{'nonstick': 1, 'vegetable': 1, 'oil': 1, 'spr...","{'cake': 1, 'nut': 2, 'dessert': 1, 'bake': 1,...","{'prehe